<a href="https://colab.research.google.com/github/fryall/fofo/blob/main/newCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import os,glob
import cv2
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # import was correct
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Dense, InputLayer


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
image_size=[244,244]
input_folder = '/content/drive/MyDrive/try1'
splitfolders.ratio(input_folder, output="ten",
                   seed=42, ratio=(.7, .3 ),
                   group_prefix=None)

Copying files: 6271 files [02:25, 43.03 files/s]


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1./255,  # Normalize pixel values
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_data = ImageDataGenerator(rescale=1./255)  # Only normalize for test data

train_path = train_data.flow_from_directory(
    "/content/ten/train",
    target_size=(224, 224),  # Resize images to match model input
    batch_size=32,
    class_mode='categorical')  # Assuming categorical classification

test_path = test_data.flow_from_directory(
    "/content/ten/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 4380 images belonging to 20 classes.
Found 1889 images belonging to 20 classes.


In [13]:
from keras.applications import VGG19
VGG=VGG19(input_shape=(224,224,3),include_top=False,weights='imagenet')

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [56]:
VGG.trainable=False

In [36]:
input_shape = (224, 224, 3)

In [63]:
model=Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(VGG)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=300,activation="relu"))
model.add(Dense(units=300,activation="relu"))
model.add(Dense(units=20,activation="softmax"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [64]:
print(VGG.output_shape)

(None, 7, 7, 512)


In [65]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [66]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 300)                 │         153,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 300)                 │          90,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 20)                  │           6,020 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,274,604 (77.34 MB)

 Trainable params: 250,220 (977.42 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [67]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\nReached 98% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [72]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.keras", monitor="val_accuracy", verbose=1,
                             save_best_only=True, save_weights_only=False)
# Removed 'early' parameter as it's not part of ModelCheckpoint

early_stopping = EarlyStopping(monitor="val_accuracy", min_delta=0, patience=40,
                               verbose=1, mode='auto')
# Created a separate EarlyStopping callback

In [ ]:
history = model.fit(x=train_path, steps_per_epoch=93, epochs=200,
                 validation_data=test_path, validation_steps=1,
                 callbacks=[checkpoint])

Epoch 1/200
 4/93 ━━━━━━━━━━━━━━━━━━━━ 40:00 27s/step - accuracy: 0.0365 - loss: 2.9676